code from https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py

In [1]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate, Conv2DTranspose, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.utils import plot_model
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load Dataset

In [2]:
data = pd.read_csv('./fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
data.Usage.value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [4]:
num_classes = 7
img_width = 48
img_height = 48

In [5]:
X = data['pixels']
y = data['emotion']

X_train = []
for i in X:
    X_train.append([int(j) for j in i.split()])

X_train = np.array(X_train)/255.0

X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_train = X_train.astype('float32')

print(X_train.shape)

(35887, 48, 48, 1)


## BiCoGAN Model

In [20]:
class BiCoGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 48
        self.img_cols = 48
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 7
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        print(self.discriminator.summary())
        plot_model(self.discriminator, show_shapes=True)
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        # noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        # img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator([z, label])

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_, label])
        valid = self.discriminator([z_, img, label])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bicogan_generator = Model([z, img, label], [fake, valid])
        self.bicogan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)

    def build_encoder(self):
        model = Sequential()

        model.add(Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=self.img_shape))
        model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Conv2D(256, (5,5), strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Conv2D(512, (5,5), strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Flatten())
        model.add(Dense(self.latent_dim))

        # model.add(Flatten(input_shape=self.img_shape))
        # model.add(Dense(512))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(512))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(self.latent_dim))

        print('encoder')
        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):

        model = Sequential()

        # # foundation for 12x12 image
        # n_nodes = 128 * 12 * 12
        # model.add(Dense(n_nodes, input_dim=self.latent_dim))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(Reshape((12, 12, 128)))
        # # upsample to 24x24
        # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        # model.add(LeakyReLU(alpha=0.2))
        # # upsample to 48x48
        # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        # model.add(LeakyReLU(alpha=0.2))
        # # generate
        # model.add(Conv2D(1, (12, 12), activation='tanh', padding='same'))

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        print('generator')
        model.summary()

        z = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([z, label_embedding])
        img = model(model_input)

        return Model([z, label], img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        d_in = concatenate([z, flat_img, label_embedding])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img, label], validity, name='discriminator')


    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        # (_, y_train), (_, _) = mnist.load_data()

        # Configure input
        # X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y.to_numpy().reshape(-1, 1)
        # y_train = y.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images and encode
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]
            z_ = self.encoder.predict(imgs)

            # Sample noise and generate img
            z = np.random.normal(0, 1, (batch_size, 100))
            imgs_ = self.generator.predict([z, labels])

            # Train the discriminator (img -> z is valid, z -> img is fake)
            d_loss_real = self.discriminator.train_on_batch([z_, imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([z, imgs_, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 7, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.bicogan_generator.train_on_batch([z, imgs, sampled_labels], [valid, fake])

            # Plot the progress
            if epoch%20 == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    # def sample_images(self, epoch):
    #     r, c = 1, 7
    #     noise = np.random.normal(0, 1, (r * c, 100))
    #     sampled_labels = np.arange(0, 7).reshape(-1, 1)

    #     gen_imgs = self.generator.predict([noise, sampled_labels])
    #     print(gen_imgs.shape)

    #     # Rescale images 0 - 1
    #     gen_imgs = 0.5 * gen_imgs + 0.5

    #     fig, axs = plt.subplots(r, c)
    #     cnt = 0
    #     for i in range(r):
    #         print(i)
    #         for j in range(c):
    #             print(j)
    #             axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
    #             axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
    #             axs[i,j].axis('off')
    #             cnt += 1
    #     fig.savefig("images/%d.png" % epoch)
    #     plt.close()

    def sample_images(self, epoch):
          r, c = 1, 7
          noise = np.random.normal(0, 1, (r * c, 100))
          sampled_labels = np.arange(0, 7).reshape(-1, 1)

          gen_imgs = self.generator.predict([noise, sampled_labels])

          # Rescale images 0 - 1
          gen_imgs = 0.5 * gen_imgs + 0.5

          fig, axs = plt.subplots(r, c)
          cnt = 0
          for j in range(c):
              axs[j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
              axs[j].set_title("label: %d" % sampled_labels[cnt])
              axs[j].axis('off')
              cnt += 1
          fig.savefig("images_3/%d.png" % epoch)
          plt.close()


In [21]:
if __name__ == '__main__':
    bicogan = BiCoGAN()
    bicogan.train(epochs=10000, batch_size=128, sample_interval=200)

Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_66 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_65 (InputLayer)           [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 1, 2304)      16128       input_66[0][0]                   
__________________________________________________________________________________________________
input_64 (InputLayer)           [(None, 100)]        0                                            
______________________________________________________________________________________